# Imports

In [1]:
import pandas as pd
train_set = pd.read_csv("csv_data/train_set_1.csv")
test_set = pd.read_csv("csv_data/test_set_1.csv")
validate_set = pd.read_csv("csv_data/test_set_1.csv")

In [2]:
import time
from transformers import AutoModel, BertTokenizerFast
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from simpletransformers.classification import ClassificationModel

# Optoimizing dataset

In [3]:
class makeDataset(Dataset):
    def __init__(self, data, max_length=100):
        self.data = data
        
        self.config = {
            "max_length": max_length,
            "padding": "max_length",
            "return_tensors": "pt",
            "truncation": True,
            "add_special_tokens": True
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        value = self.data.iloc[idx]
        return value['text'], value['label']

# Training data 2 testing data 2 final testing data 1

In [4]:
training_data = makeDataset(train_set)
train_dataloader = DataLoader(training_data, batch_size=16, shuffle=True)

In [5]:
test_data = makeDataset(test_set)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=True)

In [6]:
final_test_data = makeDataset(validate_set)
final_test_dataloader = DataLoader(final_test_data, batch_size=16, shuffle=True)

In [7]:
bert = ClassificationModel('bert', 'sagorsarker/bangla-bert-base', num_labels=5, use_cuda=True, args={
    'reprocess_input_data': True,
    'use_cached_eval_features': False,
    'overwrite_output_dir': True,
    'num_train_epochs': 2,
    'silent': True
})
bert.train_model(train_set.sample(frac=1))
torch.save(bert, "bert.pth")

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch